In [9]:
install.packages(openEBGM)

ERROR: Error in install.packages(openEBGM): object 'openEBGM' not found


In [10]:
library(openEBGM)

In [11]:
df<-read.csv('./data/product_event.csv')

In [14]:
head(df)

,Drug.Name,Metastases.to.bone,Osteonecrosis.of.jaw,Epilepsy,Stomatitis,Angioedema,Flushing,Erythema,Skin.exfoliation,Swelling,⋯,TYPE.2.DIABETES.MELLITUS,TYPE.1.DIABETES.MELLITUS,POISONING,HEPATITIS,INCORRECT.DOSE.ADMINISTERED,WITHDRAWAL.SYNDROME,DRUG.WITHDRAWAL.SYNDROME.NEONATAL,POISONING.DELIBERATE,SOMNAMBULISM,DRUG.ABUSE
,<chr>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,afinitor,240,127,20,667,28,49,102,37,63,⋯,0,0,0,0,0,0,0,0,0,0
2,avastin,89,143,61,305,0,37,158,0,0,⋯,0,0,0,0,0,0,0,0,0,0
3,carmustine,0,0,0,23,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
4,QUETIAPINE,0,0,100,31,38,0,68,24,0,⋯,65,21,120,26,36,21,20,30,26,36


In [15]:
processRaw(data=df, stratify = FALSE)

ERROR: Error in .checkInputs_processRaw(data, stratify, zeroes, list_ids): missing the appropriate column names (need 'id', 'var1', & 'var2')
